# Working with Pandas and SQL Databases (Movies Dataset)

## Creating an SQLite Database

1. __Import__ sqlite3 (as sq3) and __create__ a new SQLite Database with the name __"movies.db"__.

In [ ]:
import pandas as pd
import json
import sqlite3 as sq3

In [ ]:
#connect to the already exisitng movies
#if database doesnt exist, it will create a new database with the name "movies.db"
#also creates a connection to the database

con=sq3.connect("movies.db")
#movies.db gets added to the jupiter screen

In [ ]:
con

In [ ]:
con.execute("select * from sqlite_master").fetchall()

In [ ]:
con.execute("select name from sqlite_master where type='table' order by name").fetchall()

In [ ]:
con.close()

## Loading Data from DataFrames into an SQLite Database

2. __Load__ the json file __"some_movies.json"__ and __split__ the dataset into the following __four datasets__ (save each dataset in a Pandas DataFrame).

__Dataset #1 (Movies)__ with columns ["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]. <br>
Convert "release_date" to datetime and transform "budget" and "revenue" to Million USD before loading into the Database. 

__Dataset #2 (Votes)__ with columns ["id", "vote_count", "vote_average"]. 

__Dataset #3 (Genres)__ with columns ["genre_id", "genre_name", "id"]. <br> 

__Dataset #4 (Prod)__ with columns ["comp_id", "comp_logo_path", "comp_name", "comp_origin_country", "id" ]. <br>


3. __Load__ the datasets __into the database__ (each dataset should be a separate table in the database). __Name__ the tables "Movies", "Votes", "Genres", "Prod".

In [ ]:
with open("some_movies.json") as f:
    data=json.load(f)

In [ ]:
data

In [ ]:
df=pd.json_normalize(data,sep="_") #flatten the dataframe

In [ ]:
df

In [ ]:
df_movies=df[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()
df_movies

In [ ]:
df_movies.info()

In [ ]:
df_movies.release_date=pd.to_datetime(df.release_date)

In [ ]:
df_movies.budget=df.budget/1000000
df_movies.revenue=df.revenue/1000000

In [ ]:
df_movies

In [ ]:
df_votes=df[ ["id", "vote_count", "vote_average"]].copy()

In [ ]:
df_votes

In [ ]:
df_genres=pd.json_normalize(data=data,record_path="genres",meta="id",record_prefix="genre_")
df_genres

In [ ]:
df_prod=pd.json_normalize(data=data,record_path="production_companies",meta="id",record_prefix="comp_")
df_prod

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
con

In [ ]:
#load all four dataframes into database
df_movies.to_sql("db_movies",con,index=False)

In [ ]:
df_votes.to_sql("db_votes",con,index=False)

In [ ]:
df_genres.to_sql("db_genres",con,index=False)

In [ ]:
df_prod.to_sql("db_prod",con,index=False)

In [ ]:
con.execute("select * from sqlite_master").fetchall()

In [ ]:
con.execute("select name from sqlite_master where type='table' order by name").fetchall()

In [ ]:
con.close()

## Loading Data from SQLite Databases into DataFrames

4. __Load__ the full tables "Movies", "Votes", "Genres", "Prod" from "movies.db" into Pandas (four DataFrames). __Set__ "id" as Index. 

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
con.execute("select * from sqlite_master").fetchall()

In [ ]:
pd.read_sql("select * from db_movies",con)

In [ ]:
pd.read_sql("select * from db_movies",con).info()

In [ ]:
df_movies2=pd.read_sql("select * from db_movies",con,index_col="id",parse_dates="release_dates")

In [ ]:
df_movies2

In [ ]:
df_genres2=pd.read_sql("select * from db_genres",con,index_col="id")

In [ ]:
df_genres2

In [ ]:
con.close()

##  Some Simple SQL Queries

5. __Perform__ the following simple __SQL Queries__ and __store__ the results in DataFrames:

__Load the full "Movies" Table__.

__Load the columns "id", "revenue" and "release_date" from "Movies".__ 

__Get the Total Revenue (sum) over all movies from "Movies".__

__Count the number of Movies in "Movies".__

__Count the number of Movies that do belong to a collection.__

__Get the average budget from "Movies"__.

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM db_movies",con)

In [ ]:
pd.read_sql("SELECT * \
           FROM db_movies",con)
# \ allows to put query in new line

In [ ]:
pd.read_sql("SELECT id, revenue, release_date FROM db_movies",con)

In [ ]:
con.execute("SELECT sum(revenue) FROM db_movies").fetchall()[0][0]

In [ ]:
pd.read_sql("SELECT sum(revenue) FROM db_movies",con)

In [ ]:
#no of non missing values in the title column
pd.read_sql("SELECT count(title) FROM db_movies",con)

In [ ]:
#movies belonging to collection
pd.read_sql("SELECT count(belongs_to_collection_name) FROM db_movies",con)

In [ ]:
pd.read_sql("SELECT count(*) FROM db_movies",con)

In [ ]:
pd.read_sql("SELECT avg(budget) FROM db_movies",con)

In [ ]:
con.close()

## Some more SQL Queries

6. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Load all columns for the movie with movie id 597__.

__Load all columns for all movies with a revenue greater than 2000 (MUSD).__

__Load all columns for all movies with a revenue greater than 1500 (MUSD) and a budget below 200 (MUSD).__

__Get the minimum budget from those movies with a revenue greater than 1250 (MUSD).__

__Get all unique collection Names from "Movies".__

__Load all movies (all columns) and sort by budget from high to low.__

__Load all movies (all columns) that do not belong to a collection.__

__Load all movies (all columns) that belong to a collection.__

__Load all movies (all columns) where "Avengers..." is in the title__.

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE id=597",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE revenue>2000",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE revenue>1500 AND budget<200",con)

In [ ]:
pd.read_sql("SELECT min(budget) FROM db_movies WHERE revenue>1250",con)

In [ ]:
pd.read_sql("SELECT DISTINCT title FROM db_movies",con)
#get unique titles

In [ ]:
pd.read_sql("SELECT DISTINCT belongs_to_collection_name FROM db_movies",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies order by budget desc",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE belongs_to_collection_name IS NULL",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE belongs_to_collection_name IS NOT NULL",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies WHERE title LIKE 'Avengers%'",con)
#search movies that has following pattern in the title

In [ ]:
con.close()

## Join Queries

7. __Perform__ the following __SQL Join Queries__ and __store__ the results in DataFrames:

__Join "Movies" and "Votes"__ (all columns).

__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average").__

__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8.__

__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8 and in ascending budget order__.

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM db_movies",con)

In [ ]:
pd.read_sql("SELECT * FROM db_votes",con)

In [ ]:
pd.read_sql("SELECT * FROM db_movies JOIN db_votes ON db_movies.id=db_votes.id",con)

In [ ]:
pd.read_sql("SELECT db_movies.id,db_movies.title,db_votes.vote_average \
            FROM db_movies JOIN db_votes \
            ON db_movies.id=db_votes.id",con,index_col="id")

# \ allows to take the query to the next line 

In [ ]:
pd.read_sql("SELECT db_movies.id,db_movies.title,db_votes.vote_average \
            FROM db_movies JOIN db_votes \
            ON db_movies.id=db_votes.id \
            WHERE db_votes.vote_average>8",con,index_col="id")

In [ ]:
pd.read_sql("SELECT db_movies.id,db_movies.title,db_movies.budget,db_votes.vote_average \
            FROM db_movies JOIN db_votes \
            ON db_movies.id=db_votes.id \
            WHERE db_votes.vote_average>8 \
            ORDER BY db_movies.budget ASC",con,index_col="id")

In [ ]:
con.close()

## Final Case Study

8. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Get the Total Revenue (sum) for each Production Company.__

__Get all Production Companies for the movie "Titanic".__

__Get the Total Revenue (sum) for each Genre.__

__Get all Genres for the movie "Frozen II".__

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM db_prod",con)

In [ ]:
df2=pd.read_sql("SELECT db_prod.id,db_prod.comp_name,db_movies.revenue,db_movies.title \
              FROM db_prod LEFT JOIN db_movies \
              ON db_prod.id=db_movies.id",con)
df2
#LEFT table is db_prod RIGHT TABLE is db_movies

In [ ]:
pd.read_sql("SELECT db_prod.id,db_prod.comp_name,db_movies.revenue,db_movies.title \
              FROM db_prod INNER JOIN db_movies \
              ON db_prod.id=db_movies.id",con)
#INNER JOIN and LEFT JOIN gives te same results

In [ ]:
pd.read_sql("SELECT db_prod.id,db_prod.comp_name,db_movies.revenue,db_movies.title \
              FROM db_prod JOIN db_movies \
              ON db_prod.id=db_movies.id",con)

In [ ]:
#Get the Total Revenue (sum) for each Production Company.
df2.groupby("comp_name").revenue.sum().sort_values(ascending=False)

In [ ]:
#Get all Production Companies for the movie "Titanic"
pd.read_sql("SELECT db_prod.comp_name,db_movies.title \
            FROM db_prod LEFT JOIN db_movies \
            ON db_prod.id=db_movies.id where db_movies.title='Titanic'",con)

In [ ]:
#Get the Total Revenue (sum) for each Genre
df3=pd.read_sql("SELECT db_genres.id,db_genres.genre_name,db_movies.title,db_movies.revenue \
                FROM db_genres LEFT JOIN db_movies \
                ON db_genres.id=db_movies.id",con)
df3

In [ ]:
df3.groupby('genre_name').revenue.sum().sort_values(ascending=False)

In [ ]:
#Get all Genres for the movie "Frozen II"
pd.read_sql("select db_genres.genre_name,db_movies.title \
           FROM db_genres LEFT JOIN db_movies \
           ON db_genres.id=db_movies.id WHERE db_movies.title='Frozen II'",con)

In [ ]:
con.close()

In [ ]:
print("The End")